In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-08-29


Found rundate 2024-08-28 10:00:00 < 2024-08-29 at commit ad4fea60; returning commit 5d207fd


'2024-08-29 10:00:00'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-08-29 10:00:00, NJSP was reporting 419 YTD deaths
Current YTD Deaths (2025-08-29 10:00:01-04:00): 360
Previous year YTD Deaths (adjusted; 2024-08-29 10:00:00): 418.4073213530422
Projected 2025 total: 601.1763384852449
-14.0% change


2b4ab27f08d20bef8d11c0018fa44467acdd71ba: FAUQStats cache miss: 2024, Thu Aug 29 10:00:00 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,East Oak St,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,647,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13382,06,Cumberland,0614,Vineland City,NaN,Panther Rd,2.0,2.0,0.0,0.0,0.0,NaN,Panther Rd,2024-08-24 22:24:00-04:00
13384,09,Hudson,0907,Kearny Town,280,Interstate 280 MP 16,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-08-25 04:34:00-04:00
13385,07,Essex,0714,Newark City,NaN,West Bigelow St,1.0,1.0,0.0,0.0,0.0,NaN,West Bigelow St,2024-08-25 17:41:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,16,2,6,0,23
Bergen,12,1,4,0,17
Burlington,15,4,4,0,22
Camden,15,2,8,1,25
Cape May,7,3,0,0,9
Cumberland,8,2,3,0,12
Essex,8,3,18,1,30
Gloucester,14,4,3,0,18
Hudson,4,1,3,1,8


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

65.9% through the year, 34.1% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,19,31,16
Bergen,driver,12,14,12
Burlington,driver,20,31,15
Camden,driver,13,19,15
Cape May,driver,3,8,7
...,...,...,...,...
Salem,crashes,9,14,10
Somerset,crashes,10,18,8
Sussex,crashes,9,12,4


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,42,0,27,4,12
Bergen,28,0,14,2,11
Burlington,35,0,24,4,9
Camden,35,1,22,3,10
Cape May,23,0,16,4,0
Cumberland,21,0,12,3,8
Essex,44,1,12,10,26
Gloucester,44,0,32,14,8
Hudson,16,1,7,1,8


In [9]:
projected.to_csv(PROJECTED_CSV)